In [1]:
!pip install transformers
!pip install emoji
!pip install vncorenlp
!sudo apt-get install git-lfs
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=4b8a3e1cb9ad0ec856fcb278d1708f9825151f21bd1036515dd692c0ffcd9be4
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp



The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 93 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 0s (14.6 MB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 108827 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
Unpacking git-lfs (2.9.2-1) ...
Setting up git-lfs (2.9.

## Requirements.txt
python==3.7.12<br/>
transformers==4.20.1<br/>
emoji==2.1.0<br/>
vncorenlp==1.0.3<br/>
torch==1.11.0<br/>
matplotlib==3.5.3<br/>
numpy==1.21.6<br/>
sklearn==0.0<br/>
pandas==1.3.5<br/>
tqdm==4.64.0<br/>
regex==2021.11.10<br/>
nltk==3.7<br/>
opencv-contrib-python==4.5.4.60<br/>
opencv-python==4.5.4.60<br/>
opencv-python-headless==4.5.4.60<br/>
urllib3==1.26.12<br/>
requests==2.28.1<br/>
torchvision==0.12.0<br/>
unidecode==1.3.6<br/>

In [2]:
from torch import optim
import matplotlib.pyplot as plt
from transformers import RobertaForSequenceClassification, RobertaModel, ViTFeatureExtractor ,ViTForImageClassification
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm, notebook
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import re
import torch.nn.functional as F
from nltk.tokenize.casual import TweetTokenizer
import cv2
from urllib.request import urlopen
from nltk.tokenize.casual import TweetTokenizer
import emoji
import time
from multiprocessing import Pool
import gc
from unidecode import unidecode
from functools import partial
import ast
import requests
from torch.utils.data.dataset import random_split
from torchvision.utils import _log_api_usage_once

In [3]:
plt.style.use('ggplot')
tqdm.pandas()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
text_model_name = "vinai/phobert-base"
image_model_name = 'google/vit-base-patch16-224'

# Read the data and preprocess data

In [5]:
# paste path to train data
data = pd.read_csv('/kaggle/input/sentiment-analysis/full_train.csv')
data["Comment"].fillna('khong co gi', inplace=True)
#data['image_urls'].fillna("['https://images.foody.vn/res/g97/966781/s800/foody-anh-xoi-ngon-doi-can-921-637137229366209570.jpg']", inplace=True)
data = data.dropna()
data.reset_index(drop=True, inplace=True)
data = data.drop(['Unnamed: 0', 'RevId', 'UserId'], axis=1)
data.rename(columns={'Comment': 'text', 'Rating': 'labels'}, inplace=True)

In [6]:
# paste path to test data
test_data = pd.read_csv('/kaggle/input/sentiment-analysis/test.csv')
test_data["Comment"].fillna('khong co gi', inplace=True)
# test_data['image_urls'].fillna("['https://images.foody.vn/res/g97/966781/s800/foody-anh-xoi-ngon-doi-can-921-637137229366209570.jpg']", inplace=True)
test_data = test_data.drop(['Unnamed: 0', 'UserId'], axis=1)
test_data.rename(columns={'Comment': 'text'}, inplace=True)

In [7]:
data.columns

Index(['text', 'image_urls', 'labels'], dtype='object')

In [8]:
test_data.columns

Index(['RevId', 'text', 'image_urls'], dtype='object')

## Preprocess Text

In [9]:
!git-lfs clone https://github.com/vncorenlp/VnCoreNLP

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 236 (delta 14), reused 1 (delta 0), pack-reused 212
Receiving objects: 100% (236/236), 214.22 MiB | 27.69 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [10]:
from vncorenlp import VnCoreNLP

In [11]:
# paste path to VnCoreNLP-1.1.1.jar
rdrsegmenter = VnCoreNLP("/kaggle/working/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
tokenizer = AutoTokenizer.from_pretrained(text_model_name)
feature_extractor = ViTFeatureExtractor.from_pretrained(image_model_name)
tweet = TweetTokenizer()

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [12]:
def acronym_handling(word):
    if word in ['k','ko','khong','kh','hong','khoing']:
        return 'không'
    elif word in ['rùi','ròi','r']:
        return 'rồi'
    elif word in ['mk','mìh','mìn','mik','mh']:
        return 'mình'
    elif word in ['t','tui']:
        return 'tôi'
    elif word in ['thich','thik']:
        return 'thích'
    elif word in ['đc','dc','đươc','đuoc','đk','dk']:
        return 'được'
    elif word in ['oke','okie','okay','okela','okey','oki','okla','okiela','okelah','oklah','okelala','okila','okie rate 85']:
        return 'ok'
    elif word in ['cx','cungz','cũg','cg']:
        return 'cũng'
    elif word in ['bthg','bthuong','bth']:
        return 'bình_thường'
    elif word == 'thg':
        return 'thường'
    elif word == 'ms':
        return 'mới'
    elif word == 'nx':
        return 'nữa'
    elif word == 'ntn':
        return 'như thế nào'
    elif word in ['nhưg''nma']:
        return 'nhưng'
    elif word in ['nhìu']:
        return 'nhiều'
    elif word in ['nh','nhieu']:
        return 'nhiêu'
    elif word in ['ng','nguoi','ngươi']:
        return 'người'
    elif word in ['bâyh','bh']:
        return 'bây_giờ'
    elif word in ['j','gi']:
        return 'gì'
    elif word == 'h':
        return 'giờ'
    elif word in ['mn','mng']:
        return 'mọi_người'
    elif word in ['ord','oder','od']:
        return 'order'
    elif word == 'thui':
        return 'thôi'
    elif word in ['hn','hnoi']:
        return 'hà_nội'
    elif word == 'hnay':
        return 'hôm_nay'
    elif word in ['kp','kphai','kph']:
        return 'không_phải'
    elif word in ['ncl']:
        return 'nói_chung_là'
    return word

In [13]:
emojis = {
    '😂': 'thích', '❤': 'thích', '😍': 'thích', '👍': 'thích', '🤣': 'thích', 
    '😭': 'ghét', '😋': 'thích', '😘': 'thích', '😅': 'thích', '🥰': 'thích',
    '😁': 'thích', '🤤': 'thích', '😊': 'thích', '☺': 'thích', '😆': 'thích',
    '😌': 'thích', '👌': 'thích', '✔': 'thích', '👌🏻': 'thích', '🤗': 'thích',
    '😢': 'ghét', '🙂': 'thích', '😄': 'thích', '💯': 'thích', '😝': 'thích',
    '😉': 'thích', '✅': 'thích', '😡': 'ghét', '😀': 'thích', '💕': 'thích',
    '😑': 'ghét', '✨': 'thích', '👍🏻': 'thích','😚': 'thích','😞': 'ghét',
    '😒': 'ghét', '💋': 'thích','👏': 'thích','🤩': 'thích','♥': 'thích',
    '😃': 'thích', '👏🏻': 'thích', '💓': 'thích', '😬': 'thích', '😎': 'thích',  
    '😜': 'thích',  '😔': 'ghét', '✌': 'thích', '🥺': 'thích',
    '😕': 'ghét', '😤': 'ghét','😓': 'ghét', '😛': 'thích','👍🏼': 'thích',
    '🙁': 'ghét',  '😪': 'ghét',  '😱': 'ghét','😗': 'thích',  '😳': 'ghét',
    '🔥': 'thích', '😻': 'thích',  '😩': 'ghét', '🤪': 'thích',
    '✌': 'thích','😐': 'ghét',  '💔': 'ghét', '💖': 'thích', '😙': 'thích',
    '😹': 'thích', '💪': 'thích', '😣': 'ghét','😠': 'ghét',
    '😏': 'ghét', '😖': 'ghét',  '😇': 'thích',   '🙆': 'thích',
    '😰': 'ghét', '☑': 'thích', '😽': 'thích', '👍': 'thích',
    '💚': 'thích',  '💛': 'thích',  '🤭': 'thích',  '🥳': 'thích',
    '❎': 'ghét', '😫': 'ghét',  '🖤': 'thích',  '💁': 'thích',
    '✖': 'ghét',  '🤬': 'ghét',  '👌🏼': 'thích', '❣': 'thích',
    '💪🏻': 'thích', '😟': 'ghét', '👎': 'ghét', '😨': 'ghét',
    '👍🏿': 'thích', '😶': 'ghét', '🤧': 'ghét',  '😷': 'ghét',  '😵': 'ghét',
    '💞': 'thích','💗': 'thích',  '😿': 'ghét',  '💜': 'thích',
    '💙': 'thích', '👏🏽': 'thích',  '🙌🏻': 'thích', '🤘🏻': 'thích',
    '😯': 'thích', '💘': 'thích', '🤘': 'thích',  '🥴': 'ghét',  '😦': 'ghét',
    '🤝': 'thích', '🤢': 'ghét','🤘🏽': 'thích', '🙌': 'thích',
    '👎🏻': 'ghét', '👎🏼': 'ghét', '🖕': 'ghét', '😧': 'ghét',
    '🤮': 'ghét', '🤍': 'thích','❤️': 'thích',
}

In [14]:
def standardize_word(word):
    word = re.sub(r'(\D)\1+', r'\1', word)
    if unidecode(word[len(word)-2]) == word[len(word)-1]:
        word = word[:-1]
    for char in word:
        if char.isdigit():
            word = ''
            break
    return word


In [15]:
def process_word(word):
    word = acronym_handling(word)
    word = standardize_word(word)
    return word

In [16]:
def preprocess_fn(text, rdrsegmenter, tweet):
    text = str(text).lower()
    twte = tweet.tokenize(text)
    emo = [emojis[word] for word in twte if word in emojis]
    non_emo = [process_word(word) for word in twte if word not in emojis]
    twte = emo + non_emo
    text = ' '.join(x for x in twte)
    text = re.sub(r'[^\w\s]', '', text)
    segmenter = rdrsegmenter.tokenize(text)
    text = ''.join([' '.join(x) for x in segmenter])
    return text

In [17]:
data['text'] = data.text.progress_apply(
    partial(preprocess_fn, rdrsegmenter=rdrsegmenter, tweet=tweet),
)

100%|██████████| 9070/9070 [00:52<00:00, 172.04it/s]


In [18]:
test_data['text'] = test_data.text.progress_apply(
    partial(preprocess_fn, rdrsegmenter=rdrsegmenter, tweet=tweet),
)

100%|██████████| 5103/5103 [00:26<00:00, 195.75it/s]


## Preprocess Image

In [19]:
def fetch_url(url):
    req = urlopen(url)
    for i in range(2):
        try:
            arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
            break
        except http.client.IncompleteRead:
            if i == 1:
                raise       
    image = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    resized = cv2.resize(image, (224, 224))
    return resized

In [20]:
data['image_urls'] = [ast.literal_eval(urls)[0] for urls in data['image_urls']]
test_data['image_urls'] = [ast.literal_eval(urls)[0] for urls in test_data['image_urls']]

In [21]:
# It might be time-consuming. Around 6-7 minutes if u r lucky enough. 
def open_images(urls):
    pool = Pool(processes=10)
    results = pool.map(fetch_url, urls)
    pool.close()
    pool.join()
    return results
    
start = time.time()
data['image_urls'] = open_images(data['image_urls'])
test_data['image_urls'] = open_images(test_data['image_urls'])
print("Elapsed Time: %s" % (time.time() - start))
del start

Elapsed Time: 1345.5322239398956


In [22]:
# get texts, images and labels
my_texts = data.text.tolist()
my_labels = data.labels.tolist()
my_images = data.image_urls.tolist()
my_test_texts = test_data.text.tolist()
my_test_revid = test_data.RevId.tolist()
my_test_labels = [0] * len(my_test_texts)
my_test_images = test_data.image_urls.tolist()
del data
del test_data
gc.collect()
# print(cnt)
# print(len(train_labels) - cnt)

105

## Text_Encoding and Image_Feature_Extractor

In [23]:
class ReviewDataset(Dataset):
    def __init__(self, encodings, processed_image, labels):
        self.encodings = encodings
        self.processed_image = processed_image
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        item['processed_images'] = torch.tensor(self.processed_image[idx].get('pixel_values'))
        return item

    def __len__(self):
        return len(self.labels)

In [24]:
my_encodings = tokenizer(my_texts, truncation=True, padding=True, max_length=256)
my_test_encodings = tokenizer(my_test_texts, truncation=True, padding=True, max_length=256)
del my_texts
del my_test_texts
gc.collect()

0

In [25]:
my_image_processor = []
my_test_image_processor = []
for i in range(len(my_images)):
  tmp = feature_extractor(images=my_images[i], return_tensors="pt")
  my_image_processor.append(tmp)
del my_images
gc.collect()
for i in range(len(my_test_images)):
  tmp = feature_extractor(images=my_test_images[i], return_tensors="pt")
  my_test_image_processor.append(tmp)
del my_test_images
gc.collect()

0

## Split train and valid dataset

In [26]:
def split_train_valid(corpus, valid_ratio):
    valid_length = int(len(corpus) * valid_ratio)
    train_length = len(corpus) - valid_length
    return random_split(
        corpus, lengths=[train_length, valid_length],
    )

In [27]:
dataset = ReviewDataset(encodings=my_encodings, processed_image=my_image_processor, labels=my_labels)
test_dataset = ReviewDataset(encodings=my_test_encodings, processed_image=my_test_image_processor, labels=my_test_labels)
del my_encodings
del my_labels
del my_image_processor
del my_test_encodings
del my_test_labels
del my_test_image_processor
gc.collect()

42

In [28]:
train_dataset, valid_dataset = split_train_valid(dataset, valid_ratio=0.2)
print(len(train_dataset)) 
print(len(valid_dataset))
del dataset
gc.collect()

7256
1814


21

In [29]:
del tokenizer
del rdrsegmenter
del feature_extractor
del tweet
gc.collect()

21

In [30]:
train_loader = DataLoader(train_dataset, batch_size=8)
valid_loader = DataLoader(valid_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

## Model

In [31]:
class TextImageClassifier(torch.nn.Module):
    def __init__(self, num_labels, text_model_name, image_model_name, dropout_rate=0.25):
        super(TextImageClassifier, self).__init__()
        self.roberta = AutoModel.from_pretrained(text_model_name)
        if text_model_name == 'vinai/phobert-large':
            self.out_dim = 1024
        elif text_model_name == 'vinai/phobert-base':
            self.out_dim = 768
        self.vit = ViTForImageClassification.from_pretrained(image_model_name)
        self.vit.classifier = nn.Dropout(0)
        self.l1 = torch.nn.Linear(768 + 768, 256)
        self.l2 = torch.nn.Linear(256, num_labels)
        self.d1 = torch.nn.Dropout(0.25)
        self.d2 = torch.nn.Dropout(0.2)
        self.activation1 = torch.nn.Tanh()
        self.bn1 = torch.nn.LayerNorm(768 + 768)
        self.bn2 = torch.nn.LayerNorm(256)
        
    def forward(self, input_ids, processed_images, attention_mask):
        out1 = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        x_text = out1[1]
        x_text = self.d2(x_text)
        out2 = self.vit(processed_images)
        x_images = out2[0]
        x_images = self.d2(x_images)
        x = torch.column_stack((x_text, x_images))
        out = self.bn1(x)
        out = self.l1(out)
        out = torch.nn.Tanh()(out)
        out = self.d1(out)
        out = self.bn2(out)
        out = self.l2(out)
        output = torch.sigmoid(out)
        return output

In [32]:
model = TextImageClassifier(1, text_model_name, image_model_name)
model.to(device)

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

TextImageClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

## Training

In [33]:
num_train_epochs = 2
opt = optim.AdamW(model.parameters(), lr=1e-5)
scheduler = CosineAnnealingLR(opt, 1)
criterion = nn.BCELoss()

In [34]:
def train_epoch(model, opt, scheduler, train_loader):
    model.train()
    total_loss = total = 0

    progress_bar = notebook.tqdm(train_loader, desc='Training', leave=False)
    for batch in progress_bar:

        opt.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels']
        labels = labels.to(device)
        processed_images = batch['processed_images']
        processed_images = torch.squeeze(processed_images)
        processed_images = processed_images.to(device)
        
        outputs = model(input_ids, processed_images, attention_mask=attention_mask)

        outputs = torch.squeeze(outputs, -1)
        loss = criterion(outputs, labels)

        loss.backward()
        opt.step()
        scheduler.step()
        total_loss += loss.item()
        total += len(labels)

    # print(f"\tAUC train= {roc_auc_score(y_true, y_pred):.4f}")
    return total_loss / total



In [35]:
def validate_epoch(model, valid_loader):
    model.eval()
    y_pred, y_true = [], []
    total_loss = total = 0
    with torch.no_grad():
        progress_bar = notebook.tqdm(valid_loader, desc='Validating', leave=False)
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']
            labels = labels.to(device)
            processed_images = batch['processed_images']
            processed_images = torch.squeeze(processed_images)
            processed_images = processed_images.to(device)
        
            outputs = model(input_ids, processed_images, attention_mask=attention_mask)
        
            outputs = torch.squeeze(outputs, -1)
            loss = criterion(outputs, labels)

            logits = outputs
            labels = labels.cpu().numpy()
            logits = logits.detach().cpu().numpy()

            predictions = logits

            y_true.extend(labels)
            y_pred.extend(predictions)
            
            total_loss += loss.item()
            total += len(labels)
    
    # print(len(y_true))
    # print(len(y_pred))
    print(f"\nAUC = {roc_auc_score(y_true, y_pred):.4f}")
    return total_loss / total

In [36]:
n_epochs = 0
train_losses, valid_losses = [], []
for _ in range(num_train_epochs):
    train_loss = train_epoch(model, opt, scheduler, train_loader)
    valid_loss = validate_epoch(model, valid_loader)
    
    tqdm.write(
        f'epoch #{n_epochs + 1:3d}\ttrain_loss: {train_loss:.2e}'
        f'\tvalid_loss: {valid_loss:.2e}\n',
    )

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    n_epochs += 1

Training:   0%|          | 0/907 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Validating:   0%|          | 0/227 [00:00<?, ?it/s]


AUC = 0.9453
epoch #  1	train_loss: 3.73e-02	valid_loss: 2.78e-02



Training:   0%|          | 0/907 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Validating:   0%|          | 0/227 [00:00<?, ?it/s]


AUC = 0.9455
epoch #  2	train_loss: 2.34e-02	valid_loss: 2.73e-02



In [37]:
# for _ in range(1):
#     train_loss = train_epoch(model, opt, scheduler, train_loader)
#     valid_loss = validate_epoch(model, valid_loader)
    
#     tqdm.write(
#         f'epoch #{n_epochs + 1:3d}\ttrain_loss: {train_loss:.2e}'
#         f'\tvalid_loss: {valid_loss:.2e}\n',
#     )

#     train_losses.append(train_loss)
#     valid_losses.append(valid_loss)
    
#     n_epochs += 1

## Get the Output

In [38]:
model.eval()
pred = []    
with torch.no_grad():
    progress_bar = notebook.tqdm(test_loader, desc='Testing', leave=False)
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels']
        labels = labels.to(device)
        processed_images = batch['processed_images']
        processed_images = torch.squeeze(processed_images)
        processed_images = processed_images.to(device)
        
        outputs = model(input_ids, processed_images, attention_mask=attention_mask)
        
        outputs = torch.squeeze(outputs, -1)

        logits = outputs
        logits = logits.detach().cpu().numpy()

        pred.extend(logits)

Testing:   0%|          | 0/638 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


In [39]:
#get submission
data = {'RevId': my_test_revid,
        'Rating': pred}
submission = pd.DataFrame(data)
submission.to_csv('submission.csv', index=False)